This notebook shows how to run playoff scenarios prior to the end of regular season,
and identify how each team might win the top seed and home field advantage
throughout the playoffs. It is based on a similar notebook that runs scenarios
for divisions champs. The number of scenarios involving the entire conference
is exponentially greater than those for a division, so this sample takes about 22
minutes to run the final 2 weeks. You can shorten the script to just the last
week to make it run substantially faster (seconds not minutes).

In [4]:
from nfl import NFL
import pandas as pd
import numpy as np
import time

# for progress bars
from ipywidgets import IntProgress
from IPython.display import display

nfl = NFL().load()

In [16]:
# For reference, print conference standings not counting weeks 17 & 18
weeks = [17, 18] # more than the last week will take several minutes
conf = 'NFC'
nfl.clear(weeks)
nfl(conf)

NFC
            div overall                    division                conference  \
                    win loss tie       pct      win loss tie   pct        win   
team                                                                            
PHI    NFC-East      11    4   0  0.733333        4    1   0   0.8          7   
DAL    NFC-East      10    5   0  0.666667        4    1   0   0.8          7   
NYG    NFC-East       5   10   0  0.333333        2    3   0   0.4          4   
WAS    NFC-East       4   11   0  0.266667        0    5   0   0.0          2   
DET   NFC-North      11    4   0  0.733333        3    2   0   0.6          7   
MIN   NFC-North       7    8   0  0.466667        2    2   0   0.5          6   
GB    NFC-North       7    8   0  0.466667        2    2   0   0.5          5   
CHI   NFC-North       6    9   0       0.4        2    3   0   0.4          5   
TB    NFC-South       8    7   0  0.533333        3    1   0  0.75          6   
ATL   NFC-South       7    8   0  0.466667        3    2   0   0.6          4   
NO    NFC-South       7    8   0  0.466667        2    2   0   0.5          4   
CAR   NFC-South       2   13   0  0.133333        1    4   0   0.2          1   
SF     NFC-West      11    4   0  0.733333        5    0   0   1.0          9   
STL    NFC-West       8    7   0  0.533333        4    1   0   0.8          6   
SEA    NFC-West       8    7   0  0.533333        1    4   0   0.2          6   
ARI    NFC-West       3   12   0       0.2        0    5   0   0.0          2   

                         
     loss tie       pct  
team                     
PHI     3   0       0.7  
DAL     3   0       0.7  
NYG     6   0       0.4  
WAS     8   0       0.2  
DET     3   0       0.7  
MIN     4   0       0.6  
GB      5   0       0.5  
CHI     5   0       0.5  
TB      4   0       0.6  
ATL     6   0       0.4  
NO      6   0       0.4  
CAR    10   0  0.090909  
SF      1   0       0.9  
STL     4   0       0.6  
SEA     5   0  0.545455  
ARI     8   0       0.2

In [119]:
# Find conference leaders based on record; eliminate those not
# in the running

z = nfl(conf).standings.sort_values(('overall','pct'), ascending=False)
teams = set(z.loc[z[('overall','win')]>=z.iloc[0][('overall','win')]-len(weeks)].index)

z.loc[list(teams)]

div overall                    division                     \
                    win loss tie       pct      win loss tie       pct   
team                                                                     
DAL    NFC-East      10    5   2  0.647059        4    1   1      0.75   
SF     NFC-West      11    4   2  0.705882        5    0   1  0.916667   
PHI    NFC-East      11    4   2  0.705882        4    1   1      0.75   
DET   NFC-North      11    4   2  0.705882        3    2   1  0.583333   

     conference                     
            win loss tie       pct  
team                                
DAL           7    3   2  0.666667  
SF            9    1   2  0.833333  
PHI           7    3   2  0.666667  
DET           7    3   2  0.666667

In [18]:
# print the relevant schedule for reference. It's helpful to know
# who plays who
nfl.schedule(teams, weeks, by='game')

at hscore ascore
week ht                    
17   DAL  DET   None   None
     PHI  ARI   None   None
     WAS   SF   None   None
18   DET  MIN   None   None
     NYG  PHI   None   None
     SF   STL   None   None
     WAS  DAL   None   None

In [19]:
# iterate over all possible outcomes and count the number
# of times each team wins the division.
# this can take 15 minutes or more in fast mode for a 2-week span

results = pd.DataFrame(columns=pd.MultiIndex.from_product([weeks, teams], names=['week','team']))
results.index.name = 'scenario'
results[('result','outcome')] = np.nan
results[('result','rule')] = np.nan

# calculate the number of scenarios to set up the progress bar
# it's the number of potential outcomes (win,loss.tie = 3)
# to the power of the number of relevant games (7 in this case)
# pow(3, 7) = 2187

bar = IntProgress(min=0, max=3 ** len(nfl.schedule(teams, weeks, by='game')))
display(bar)

start = time.time()
for elem in nfl.scenarios(weeks, teams):
    nfl.clear(weeks)
    nfl.set(elem)

    sch = nfl.schedule(teams, weeks)

# NB: if you are only interested in specific scenarios, e.g.
# ones where a particuar team wins all their remaining games
# you can typically save a lot of time by querying the resulting
# schedule for the current scenario at this point and avoid
# the call to tiebreaks, which even in fast mode can be
# very time consuming
    
    t = nfl.tiebreaks(teams, fast=True)

    z = len(results)
    results.loc[z] = sch['wlt']
    results.loc[z, ('result','outcome')] = t.index[0]
    if len(t) > 1:
        results.loc[z, ('result','rule')] = t.iloc[1]
    else:
        results.loc[z, ('result','rule')] = 'overall'

    bar.value += 1

bar.layout.display = 'none'
print('Elapsed time: {}'.format(time.time() - start))
results

IntProgress(value=0, max=2187)

Elapsed time: 1329.5075898170471


week       17                     18                   result  \
team      DAL    SF  PHI   DET   DAL    SF   PHI  DET outcome   
scenario                                                        
0         win  loss  win  loss  loss   win  loss  win      SF   
1         win  loss  win  loss   win   win  loss  win      SF   
2         win  loss  win  loss   tie   win  loss  win      SF   
3         win  loss  win  loss  loss  loss  loss  win     PHI   
4         win  loss  win  loss   win  loss  loss  win     DAL   
...       ...   ...  ...   ...   ...   ...   ...  ...     ...   
2182      tie   tie  tie   tie   win  loss   tie  tie     PHI   
2183      tie   tie  tie   tie   tie  loss   tie  tie     PHI   
2184      tie   tie  tie   tie  loss   tie   tie  tie      SF   
2185      tie   tie  tie   tie   win   tie   tie  tie      SF   
2186      tie   tie  tie   tie   tie   tie   tie  tie      SF   

week                        
team                  rule  
scenario                    
0             head-to-head  
1             head-to-head  
2             head-to-head  
3         victory-strength  
4             head-to-head  
...                    ...  
2182      victory-strength  
2183      victory-strength  
2184          head-to-head  
2185          head-to-head  
2186          head-to-head  

[2187 rows x 10 columns]

In [86]:
# Sanity check that none of the scenarios depend on ranks or
# netpoints; those would require scenarios with specific
# point spreads
results.groupby(('result','rule')).count()

week                17                      18                    result
team               DAL    SF   PHI   DET   DAL    SF   PHI   DET outcome
(result, rule)                                                          
common-games       102   102   102   102   102   102   102   102     102
conference         216   216   216   216   216   216   216   216     216
division            13    13    13    13    13    13    13    13      13
head-to-head       336   336   336   336   336   336   336   336     336
overall           1448  1448  1448  1448  1448  1448  1448  1448    1448
victory-strength    72    72    72    72    72    72    72    72      72

In [87]:
# This shows how many scenarios result in each team winning the division
results.groupby(('result','outcome')).count()

week                17                  18                result
team               DAL   SF  PHI  DET  DAL   SF  PHI  DET   rule
(result, outcome)                                               
DAL                 63   63   63   63   63   63   63   63     63
DET                580  580  580  580  580  580  580  580    580
PHI                550  550  550  550  550  550  550  550    550
SF                 994  994  994  994  994  994  994  994    994

In [60]:
# It looks like SF is in the best position. See if they are
# "in control of their destiny," i.e. they get the top seed so
# long as they win
results[(results.xs('SF',level=1,axis=1) == 'win').all(axis=1)]. \
    groupby(('result','outcome')).count()

week                17                  18                result
team               DAL   SF  PHI  DET  DAL   SF  PHI  DET   rule
(result, outcome)                                               
SF                 243  243  243  243  243  243  243  243    243

In [41]:
# Now do the same for DET
results[(results.xs('DET',level=1,axis=1) == 'win').all(axis=1)]. \
    groupby(('result','outcome')).count()

week                17                  18                result
team               DAL   SF  PHI  DET  DAL   SF  PHI  DET   rule
(result, outcome)                                               
DET                216  216  216  216  216  216  216  216    216
SF                  27   27   27   27   27   27   27   27     27

In [62]:
# DET does not control its destiny because there are
# scenarios where SF still gets the top seed even if 
# DET wins every game. It may require at least one
# non-win by SF, so test that
results[(results.xs('DET',level=1,axis=1) == 'win').all(axis=1) &
    (results.xs('SF',level=1,axis=1) != 'win').any(axis=1)]. \
    groupby(('result','outcome')).count()

week                17                  18                result
team               DAL   SF  PHI  DET  DAL   SF  PHI  DET   rule
(result, outcome)                                               
DET                216  216  216  216  216  216  216  216    216

In [65]:
# PHI needs at least one non-win by SF and one by DET

results[(results.xs('PHI',level=1,axis=1) == 'win').all(axis=1) & 
    (results.xs('SF',level=1,axis=1) != 'win').any(axis=1) & 
    (results.xs('DET',level=1,axis=1) != 'win').any(axis=1)].groupby(('result','outcome')).count()

week                17                  18                result
team               DAL   SF  PHI  DET  DAL   SF  PHI  DET   rule
(result, outcome)                                               
PHI                192  192  192  192  192  192  192  192    192

In [79]:
# SF can still get the top seed with only 1 win or tie, provided
# DET and PHI each lose at least once
results[(results.xs('SF',level=1,axis=1) != 'loss').all(axis=1)
    & (results.xs('DET',level=1,axis=1) == 'loss').any(axis=1)
    & (results.xs('PHI',level=1,axis=1) == 'loss').any(axis=1)
    ].groupby(('result','outcome')).count()

week                17                  18                result
team               DAL   SF  PHI  DET  DAL   SF  PHI  DET   rule
(result, outcome)                                               
SF                 300  300  300  300  300  300  300  300    300